In [136]:
import numpy as np
import pandas as pd

NEW_COLUMN_NAME = "clamped_annotations"
MAX_X = 1280
MAX_Y = 720

In [145]:
def extract(obj, key):
    if isinstance(obj, dict):
        return obj[key]
    else:
        return obj


def build_annotation(row):
    if pd.isna(row["annotations"]):
        return np.nan

    return {"x": int(row["x"]), "y": int(row["y"]), "width": int(row["width"]), "height": int(row["new_height"])}


def remove_nan_lists(x):
    if pd.isna(x[0]):
        return []
    else:
        return x


def create_new_annotations(df):
    # read and explode
    df["annotations"] = df["annotations"].apply(eval)

    df = df.explode("annotations")
    
    # extract x, y, width, height into seperate columns
    df["x"] = df["annotations"].apply(extract, key="x")
    df["y"] = df["annotations"].apply(extract, key="y")
    df["width"] = df["annotations"].apply(extract, key="width")
    df["height"] = df["annotations"].apply(extract, key="height")

    df["x2"] = df["x"] + df["width"]
    df["y2"] = df["y"] + df["height"]

    # create the clamped new coordinates
    df["clamped_x2"] = np.minimum(df["x2"], MAX_X)
    df["clamped_y2"] = np.minimum(df["y2"], MAX_Y)
    
    # for statistics only
    df["new_area"] = (df.clamped_x2 - df.x) * (df.clamped_y2-df.y)
    df["old_area"] = (df.x2 - df.x) * (df.y2-df.y)
    df["area_share"] = df["new_area"] / df["old_area"]
    assert np.all((df["area_share"]>0.3) | df["area_share"].isna())

    df["new_height"] = df["clamped_y2"] - df["y"]
    df["new_width"] = df["clamped_x2"] - df["x"]
    assert np.all((df.new_height <= df.height) | df.height.isna())
    assert np.all((df.new_width <= df.width) | df.width.isna())


    df[NEW_COLUMN_NAME] = df.apply(build_annotation, axis=1)

    # restore the structure with a new column: new_annotations
    new_df = df[["video_id", "sequence", "video_frame", "sequence_frame", "image_id", "annotations", NEW_COLUMN_NAME]]
    new_df = new_df\
        .groupby("image_id")\
        .agg({"video_id": "first", "sequence": "first", "video_frame": "first", "sequence_frame": "first", "annotations": list, NEW_COLUMN_NAME: list})\
        .reset_index()[["video_id", "sequence", "video_frame", "sequence_frame", "image_id", "annotations", NEW_COLUMN_NAME]]


    new_df["annotations"] = new_df["annotations"].apply(remove_nan_lists)
    new_df[NEW_COLUMN_NAME] = new_df[NEW_COLUMN_NAME].apply(remove_nan_lists)

    return new_df.sort_values(["video_id", "video_frame"]).reset_index(drop=True)

In [149]:
for file in ["train_mini.csv", "val_mini.csv"]:
    df = pd.read_csv(file)
    new_df = create_new_annotations(df)
    changed_lines = new_df[new_df.annotations != new_df[NEW_COLUMN_NAME]]
    print(f"{file}: {len(changed_lines)} frames got clamped")
    new_df.to_csv(f"{file}", index=False)

train_mini.csv: 4 frames got clamped
val_mini.csv: 0 frames got clamped
